# Building custom training loops

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from ipynb.fs.full.Useful_funcs import data_pipeline, pre_model, create_huber # Custom funcs for data processing, modelling, compiling and training
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import selu, relu, elu
from tensorflow.keras.initializers import lecun_normal, he_normal
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import mse

## Loading the dataset

In [ ]:
housing = fetch_california_housing()
x_train, x_train_scaled, x_valid, x_valid_scaled, x_test, x_test_scaled, y_train, y_valid, y_test = 

## Building the model

In [2]:
pre_model()

- We will first build a simple model. Need not compile it as we will be handling the training manually.

In [ ]:
model = keras.